In [1]:
import collections
import gzip
import io
import pymongo
import rdflib
import urllib.request

from mosmo.knowledge import kb
from mosmo.model import DS, Molecule, Reaction, KbEntry, DbXref

KB = kb.configure_kb()

# Load ChEBI data verbatim
- DB dump flat-files hosted at https://ftp.ebi.ac.uk/pub/databases/chebi/Flat_file_tab_delimited/
- Some files are gzipped, some are not
- Use TextIOWrapper to tolerate malformed utf-8, which has been a problem at times
- Use the full set of compounds, not just 3-star. But, do skip obsolete and replaced entries.

In [2]:
def download_tsv(filename):
    url = f'https://ftp.ebi.ac.uk/pub/databases/chebi/Flat_file_tab_delimited/{filename}.tsv'
    request = urllib.request.urlopen(url)
    return io.TextIOWrapper(request, errors='replace')

def download_tsv_gz(filename):
    url = f'https://ftp.ebi.ac.uk/pub/databases/chebi/Flat_file_tab_delimited/{filename}.tsv.gz'
    request = urllib.request.urlopen(url)
    gunz = gzip.GzipFile(fileobj=request, mode='rb')
    return io.TextIOWrapper(gunz, errors='replace')

def file_contents(f):
    """Assumes a tab-delimited text file with a single header row. Yields one dict per data row."""
    line = f.readline().strip()
    header = line.split('\t')
    rownum = 0
    while line:
        line = f.readline().strip()
        if line:
            fields = line.split('\t')
            if len(fields) == len(header):
                yield {k: v for k, v in zip(header, fields)}
            else:
                raise ValueError(f"Malformed data at row {rownum}: '{line}'")
            rownum += 1

### Assemble each Molecule with name, synonyms, mass, charge, formula, and InChI string

In [3]:
%%time
compounds = {}
status = collections.Counter()
for row in file_contents(download_tsv_gz('compounds')):
    status.update({row['STATUS']: 1})
    if row['STATUS'] in ['C', 'S'] and row['PARENT_ID'] == 'null':
        compounds[row['ID']] = Molecule(id=row['ID'], db=DS.CHEBI, name=row['NAME'])
for s, count in status.items():
    print(f'  {s}: {count}')
print(f'{len(compounds)} compounds extracted')

  C: 79510
  D: 1844
  E: 61255
  S: 84821
146369 compounds extracted
CPU times: user 2.44 s, sys: 40.9 ms, total: 2.48 s
Wall time: 8.83 s


In [4]:
%%time
compound_names = collections.defaultdict(set)
for row in file_contents(download_tsv_gz('names')):
    if row['COMPOUND_ID'] in compounds:
        compound_names[row['COMPOUND_ID']].add(row['NAME'])
for compound_id, names in compound_names.items():
    compound = compounds[compound_id]
    compound.aka = list(names - {compound.name})
print(f'{len(compound_names)} have additional names.')

141643 have additional names.
CPU times: user 3.04 s, sys: 107 ms, total: 3.15 s
Wall time: 11.1 s


In [5]:
%%time
dtypes = collections.Counter()
updated = set()
for row in file_contents(download_tsv('chemical_data')):
    if row['COMPOUND_ID'] in compounds:
        dtype = row['TYPE']
        dtypes.update({dtype: 1})
        if dtype == 'MASS':
            compounds[row['COMPOUND_ID']].mass = float(row['CHEMICAL_DATA'])
            updated.add(row['COMPOUND_ID'])
        elif dtype == 'CHARGE':
            compounds[row['COMPOUND_ID']].charge = int(row['CHEMICAL_DATA'])
            updated.add(row['COMPOUND_ID'])
        elif dtype == 'FORMULA':
            compounds[row['COMPOUND_ID']].formula = row['CHEMICAL_DATA']
            updated.add(row['COMPOUND_ID'])

print(f'Chemical data added to {len(updated)} compounds:')
for dtype, count in dtypes.items():
    print(f'{dtype:>8s}: {count}')

Chemical data added to 138032 compounds:
 FORMULA: 138640
    MASS: 137135
  CHARGE: 137918
MONOISOTOPIC MASS: 137134
CPU times: user 4.66 s, sys: 102 ms, total: 4.76 s
Wall time: 20.9 s


In [6]:
%%time
inchis = 0
for row in file_contents(download_tsv('chebiId_inchi')):
    if row['CHEBI_ID'] in compounds:
        inchis += 1
        compounds[row['CHEBI_ID']].inchi = row['InChI']
        
print(f'{inchis} InChI strings')

126895 InChI strings
CPU times: user 1.6 s, sys: 273 ms, total: 1.87 s
Wall time: 1min 32s


### Take cross-references from key sources

In [7]:
%%time
db_mapping = {
    'CAS Registry Number': DS.CAS,
    'KEGG COMPOUND accession': DS.KEGG,
    'KEGG GLYCAN accession': DS.KEGG,
    'KEGG DRUG accession': DS.KEGG,
    'MetaCyc accession': DS.METACYC,
    'LINCS accession': DS.LINCS,
    'Wikipedia accession': DS.WIKI,
}
db_count = collections.defaultdict(int)

compound_xrefs = collections.defaultdict(set)
for row in file_contents(download_tsv('database_accession')):
    if row['COMPOUND_ID'] in compounds and row['TYPE'] in db_mapping:
        db = db_mapping[row['TYPE']]
        db_count[db] += 1
        compound_xrefs[row['COMPOUND_ID']].add(DbXref(id=row['ACCESSION_NUMBER'], db=db))

for compound_id, xrefs in compound_xrefs.items():
        compounds[compound_id].xrefs = xrefs

print(f'XRef usage across {len(compound_xrefs)} compounds:')
for db in sorted(db_count.keys()):
    print(f'{db.id:>12}: {db_count[db]}')


XRef usage across 32484 compounds:
         CAS: 32789
        KEGG: 15970
       LINCS: 1691
     METACYC: 6683
        WIKI: 6028
CPU times: user 2.5 s, sys: 119 ms, total: 2.61 s
Wall time: 23.9 s


## Uncomment this cell to write ChEBI to the DB

In [8]:
# %%time
# collection = KB.client[KB.CHEBI.client_db][KB.CHEBI.collection]
# collection.drop()
# for compound in compounds.values():
#     KB.put(KB.CHEBI, compound, bypass_cache=True)
# collection.create_index('name', name='name', collation=pymongo.collation.Collation(locale='en_US', strength=1))
# collection.create_index('aka', name='aka', collation=pymongo.collation.Collation(locale='en_US', strength=1))
# collection.create_index([('xrefs.id', pymongo.ASCENDING), ('xrefs.db', pymongo.ASCENDING)],
#                         name='xrefs',
#                         collation=pymongo.collation.Collation(locale='en_US', strength=1))
# print("success")

success
CPU times: user 30.2 s, sys: 2.35 s, total: 32.5 s
Wall time: 52.2 s


In [9]:
KB.find(KB.CHEBI, 'ribose')

[[33942] ribose,
 [27476] beta-D-ribopyranose,
 [45506] alpha-D-ribose,
 [47013] D-ribofuranose]

In [10]:
KB.find(KB.CHEBI, 'ribose')[-1].__dict__

{'id': '47013',
 'db': Datasource(id='CHEBI', name='Chemical Entities of Biological Interest (ChEBI)', home='https://www.ebi.ac.uk/chebi/', urlpat={<class 'mosmo.model.core.Molecule'>: 'http://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:{id}'}),
 'name': 'D-ribofuranose',
 'shorthand': None,
 'description': None,
 'aka': ['D-Ribose',
  'D-ribose',
  'ribose',
  'WURCS=2.0/1,1,0/[a222h-1x_1-4]/1/',
  '(3R,4S,5R)-5-(hydroxymethyl)tetrahydrofuran-2,3,4-triol'],
 'xrefs': {CAS:50-69-1, CAS:613-83-2, KEGG:C00121},
 'formula': 'C5H10O5',
 'mass': 150.1299,
 'charge': 0,
 'inchi': 'InChI=1S/C5H10O5/c6-1-2-3(7)4(8)5(9)10-2/h2-9H,1H2/t2-,3-,4-,5?/m1/s1',
 'variations': None,
 'canonical_form': None,
 'default_form': None}

# Load RHEA master reactions verbatim

RHEA is organized around 'quartets'
- Master - indeterminate or unspecified direction
- irreversible left -> right
- irreversible right -> left
- explicitly reversible

Not clear what is gained by this representation vs say a reversibility attribute. Maybe it's all about the cross-references to other reaction DBs.

## RHEA reaction definitions are in RDF
- Hosted at [ftp.expasy.org](https://ftp.expasy.org/databases/rhea/rdf/rhea.rdf.gz) (see [rhea-db.org/help/download](https://www.rhea-db.org/help/download#Reactions))

In [11]:
%%time
request = urllib.request.urlopen("https://ftp.expasy.org/databases/rhea/rdf/rhea.rdf.gz")
rhea_rdf = rdflib.Graph().parse(gzip.GzipFile(fileobj=request), format="application/rdf+xml")

RDFS = rdflib.namespace.RDFS
RH = rdflib.namespace.Namespace('http://rdf.rhea-db.org/')
reaction_ids = list(rhea_rdf.subjects(RDFS.subClassOf, RH.Reaction))
print(f"This version of RHEA has {len(reaction_ids)} primary reactions.")

# Use built-in namespace manager for RDFS and rhea itself (mainly predicates)
rhea_rdf.bind('RH', RH)
rhea_rdf.bind('RDFS', RDFS)

# For everything else, use namespaces to map terms directly to xrefs.
nsmap = [
    (rdflib.namespace.Namespace('http://purl.obolibrary.org/obo/CHEBI_'), DS.CHEBI),
    (rdflib.namespace.Namespace('http://purl.uniprot.org/enzyme/'), DS.EC),
    (rdflib.namespace.Namespace('http://purl.obolibrary.org/obo/GO_'), DS.GO),
    (rdflib.namespace.Namespace('http://identifiers.org/kegg.reaction/'), DS.KEGG),
    (rdflib.namespace.Namespace('http://identifiers.org/biocyc/ECOCYC:'), DS.ECOCYC),
    (rdflib.namespace.Namespace('http://identifiers.org/biocyc/METACYC:'), DS.METACYC),
    (rdflib.namespace.Namespace('http://purl.uniprot.org/core/'), DS.get("UNIPROT")),
    (rdflib.namespace.Namespace('http://identifiers.org/macie/'), DS.MACIE),
    (rdflib.namespace.Namespace('http://rdf.ncbi.nlm.nih.gov/pubmed/'), DS.get("PUBMED")),
    (rdflib.namespace.Namespace('http://identifiers.org/reactome/'), DS.REACT),
]

def uri_to_xref(uri):
    for ns, db in nsmap:
        if uri in ns:
            return DbXref(db=db, id=uri.removeprefix(ns))
    return uri

This version of RHEA has 17250 primary reactions.
CPU times: user 1min 25s, sys: 968 ms, total: 1min 26s
Wall time: 1min 31s


## Explore the structure a bit

In [12]:
def show(s):
    for p, o in rhea_rdf[s]:
        if type(o) == rdflib.Literal:
            print((p.n3(rhea_rdf.namespace_manager), o.toPython()))
        elif o in rhea_rdf.namespace_manager:
            print((p.n3(rhea_rdf.namespace_manager), o.n3(rhea_rdf.namespace_manager)))
        else:
            print((p.n3(rhea_rdf.namespace_manager), uri_to_xref(o)))

In [13]:
show(RH["16109"])

('RDFS:subClassOf', 'RH:Reaction')
('RH:id', 16109)
('RH:accession', 'RHEA:16109')
('RDFS:label', 'ATP + beta-D-fructose 6-phosphate = ADP + beta-D-fructose 1,6-bisphosphate + H(+)')
('RH:equation', 'ATP + beta-D-fructose 6-phosphate = ADP + beta-D-fructose 1,6-bisphosphate + H(+)')
('RH:htmlEquation', 'ATP + &#946;-<small>D</small>-fructose 6-phosphate = ADP + &#946;-<small>D</small>-fructose 1,6-bisphosphate + H<small><sup>+</sup></small>')
('RH:status', 'RH:Approved')
('RH:directionalReaction', 'RH:16110')
('RH:directionalReaction', 'RH:16111')
('RH:bidirectionalReaction', 'RH:16112')
('RH:isChemicallyBalanced', True)
('RH:isTransport', False)
('RH:ec', EC:2.7.1.11)
('RDFS:seeAlso', GO:0003872)
('RH:citation', PUBMED:4224472)
('RH:citation', PUBMED:4248230)
('RH:citation', PUBMED:12981037)
('RH:citation', PUBMED:23729568)
('RH:citation', PUBMED:26205495)
('RH:citation', PUBMED:4237772)
('RH:side', 'RH:16109_L')
('RH:side', 'RH:16109_R')


In [14]:
show(RH["16109_L"])

('RDFS:subClassOf', 'RH:ReactionSide')
('RH:curatedOrder', 1)
('RH:transformableTo', 'RH:16109_R')
('RH:contains', 'RH:Participant_16109_compound_6372')
('RH:contains', 'RH:Participant_16109_compound_5371')
('RH:contains1', 'RH:Participant_16109_compound_6372')
('RH:contains1', 'RH:Participant_16109_compound_5371')


In [15]:
show(RH["Participant_16109_compound_6372"])

('RH:compound', 'RH:Compound_6372')
('RDFS:subClassOf', 'RH:Compound_6372')
('RDFS:subClassOf', 'RH:ReactionParticipant')


In [16]:
show(RH["Compound_6372"])

('RH:id', 6372)
('RH:accession', 'CHEBI:30616')
('RH:name', 'ATP')
('RH:htmlName', 'ATP')
('RH:formula', 'C10H12N5O13P3')
('RH:charge', -4)
('RDFS:subClassOf', 'RH:SmallMolecule')
('RDFS:subClassOf', CHEBI:30616)
('RH:chebi', CHEBI:30616)


## Extract all Compounds to corresponding ChEBI entries
- RH.SmallMolecule: RH.chebi predicate points to a CHEBI xref
- RH.Polymer: RH.underlyingChebi predicate points to a CHEBI xref
- RH.GenericCompound subclasses have 1 or more RH.reactivePart, each with an RH.chebi

In [17]:
%%time
rhea_compound = {}  # key=compound_id, value={molecule: count}
chebi_not_found = set()
small_mols = 0
for compound_id in rhea_rdf.subjects(RDFS.subClassOf, RH.SmallMolecule):
    ref = uri_to_xref(rhea_rdf.value(compound_id, RH.chebi).toPython())
    mol = KB.deref(ref)
    if mol is not None:
        rhea_compound[compound_id] = {mol: 1}
        small_mols += 1
    else:
        chebi_not_found.add(ref)

polymers = 0
for compound_id in rhea_rdf.subjects(RDFS.subClassOf, RH.Polymer):
    ref = uri_to_xref(rhea_rdf.value(compound_id, RH.underlyingChebi).toPython())
    mol = KB.deref(ref)
    if mol is not None:
        rhea_compound[compound_id] = {mol: 1}
        polymers += 1
    else:
        chebi_not_found.add(ref)

generics = collections.Counter()
generic_ids = []
for gtype in rhea_rdf.subjects(RDFS.subClassOf, RH.GenericCompound):
    generic_ids.extend(rhea_rdf.subjects(RDFS.subClassOf, gtype))
for compound_id in generic_ids:
    refs = []
    for part in rhea_rdf.objects(compound_id, RH.reactivePart):
        refs.append(uri_to_xref(rhea_rdf.value(part, RH.chebi).toPython()))
            
    mols = [KB.deref(ref) for ref in refs]
    if all(mol is not None for mol in mols):
        rhea_compound[compound_id] = dict(collections.Counter(mols))
        generics.update({len(mols): 1})
    else:
        for ref, mol in zip(refs, mols):
            if mol is None:
                chebi_not_found.add(ref)

print(f'{len(rhea_compound)} Compound entries resloved'
      f' ({small_mols} CHEBI, {polymers} POLYMER, {sum(generics.values())} GENERIC)')
print(f'GENERIC molecule reactive part count:')
for k in sorted(generics.keys()):
    print(f'    {k}: {generics[k]}')
print()
print(f'{len(chebi_not_found)} referenced ChEBI entries were not found.')


13962 Compound entries resloved (11891 CHEBI, 259 POLYMER, 1812 GENERIC)
GENERIC molecule reactive part count:
    1: 1732
    2: 72
    3: 6
    5: 2

10 referenced ChEBI entries were not found.
CPU times: user 4.54 s, sys: 386 ms, total: 4.93 s
Wall time: 7.46 s


The missing ChEBI entries all have a status of 'E', i.e. it exists but has not progressed beyond that. Given the small number, vs the large number of 'E' entries that would clutter up our KB, we will skip these.

## Extract all Reactions

In [18]:
# Simplify funky containsXXX predicates. But ignore N, 2n, etc
contains_count = {}
for verb in rhea_rdf.subjects(RDFS.subPropertyOf, RH.contains):
    count = rhea_rdf.value(verb, RH.coefficient).toPython()
    if count.isdigit():
        contains_count[verb] = int(count)
    else:
        contains_count[verb] = None

def extract_stoichiometry(rxn_id):
    stoichiometry = collections.Counter()
    for side_id in rhea_rdf.objects(rxn_id, RH.side):
        multiplier = -1 if rhea_rdf.value(side_id, RH.curatedOrder).toPython() == 1 else 1
        for p, o in rhea_rdf[side_id]:
            if p in contains_count:
                count = contains_count[p]
                if count is None:
                    raise ValueError(f"<STOICH> {rxn_id.n3(rhea_rdf.namespace_manager)} has unsupported count")

                compound_id = rhea_rdf.value(o, RH.compound)
                if compound_id in rhea_compound:
                    stoichiometry.update({
                        k: v * multiplier
                        for k, v in rhea_compound[compound_id].items()
                    })
                else:
                    raise ValueError(f"<NOCHEBI> {rxn_id.n3(rhea_rdf.namespace_manager)} references missing ChEBI ids")
    return dict(stoichiometry)

def rxn_xrefs(rxn_id):
    if rxn_id is None:
        return
    for ec in rhea_rdf.objects(rxn_id, RH.ec):
        xref = uri_to_xref(ec)
        if type(xref) == DbXref:
            yield xref
        else:
            raise ValueError(f"<EC> {rxn_id} has unexpected EC value: {ec}")
    for seealso in rhea_rdf.objects(rxn_id, RDFS.seeAlso):
        xref = uri_to_xref(seealso)
        if type(xref) == DbXref:
            yield xref
        else:
            raise ValueError(f"<SEEALSO> {rxn_id} has unexpected seeAlso: {seealso}")
    
def extract_reaction(rxn_id):
    stoichiometry = extract_stoichiometry(rxn_id)

    # Lump together all of the quartet's xrefs -- not rigorous by RHEA standards, but reasonable for us.
    xrefs = set(rxn_xrefs(rxn_id))
    for subrxn in rhea_rdf.objects(rxn_id, RH.directionalReaction):
        xrefs.update(rxn_xrefs(subrxn))
    xrefs.update(rxn_xrefs(rhea_rdf.value(rxn_id, RH.bidirectionalReaction)))

    # RHEA does not provide nice names on its own; get the name from EC if possible
    ec_nums = [xref for xref in xrefs if xref.db == DS.EC]
    if ec_nums:
        ec_entry = KB.deref(ec_nums[0])
        if ec_entry is None:
            raise ValueError(f"<EC> EC entry {ec_nums[0]} is missing")

        name = ec_entry.name
        # Mark names as ambiguous where there are multiple ECs.
        if len(ec_nums) > 1:
            name = '* ' + name
    else:
        # Fallback: use the RHEA-provided label, which is just the reaction formula.
        name = rhea_rdf.value(rxn_id, RDFS.label).toPython()

    # Heuristic: if the bidirectional reaction has a cross-ref, or both the directional ones do, we infer reversibility
    reversible = (
        (rhea_rdf.value(rhea_rdf.value(rxn_id, RH.bidirectionalReaction), RDFS.seeAlso) is not None) or
        all(rhea_rdf.value(dir_id, RDFS.seeAlso) is not None
            for dir_id in rhea_rdf.value(rxn_id, RH.directionalReaction))
    )

    return Reaction(
        id = rxn_id.removeprefix(RH),
        db = DS.RHEA,
        name = name,
        xrefs = xrefs or None,
        stoichiometry = stoichiometry,
        reversible = reversible,
    )

In [19]:
%%time
rxns = {}
skipped = collections.Counter()
fail = collections.defaultdict(dict)

for rxn_id in reaction_ids:
    if rhea_rdf.value(rxn_id, RH.status) == RH.Approved:
        try:
            rxn = extract_reaction(rxn_id)
            rxns[rxn.id] = rxn
        except ValueError as e:
            group, _ = e.args[0].split(maxsplit=1)
            fail[group][rxn_id] = e
    else:
        skipped.update({rhea_rdf.value(rxn_id, RH.status).n3(rhea_rdf.namespace_manager): 1})

print(f"{len(rxns)} of {len(reaction_ids)} reactions successfully extracted.")
print("Failures:")
for group, fails in fail.items():
    print(f"  {group:10}: {len(fails):>5d}")
print()
print("Skipped:")
for status, count in skipped.items():
    print(f"  {status}: {count}")


16766 of 17250 reactions successfully extracted.
Failures:
  <STOICH>  :    75
  <NOCHEBI> :    16

Skipped:
  RH:Obsolete: 280
  RH:Preliminary: 113
CPU times: user 6.66 s, sys: 125 ms, total: 6.78 s
Wall time: 8.22 s


### Failure Summary:
- \<NOCHEBI>: Reactions referencing any of the missing ChEBI entries discussed above.
- \<STOICH>: Reactions using stoichiometries such as N, Nplus1, 2n, etc. These will likely be relevant for cases such as fatty-acid biosynthesis. Defer until we have a systematic way to represent them in the knowledge base.
    - A pragmatic approach might be to set n=2 (so n, n-1, n+1, and 2n are all valid), and then mark in the description that we did this. Or n=100, which puts it outside the range of all the other 'contains' predicates.
- Skipped: Working as intended.

Acceptable.

## Uncomment this cell to write RHEA to the DB

In [20]:
# %%time
# collection = KB.client[KB.RHEA.client_db][KB.RHEA.collection]
# collection.drop()
# for reaction in rxns.values():
#     KB.put(KB.RHEA, reaction, bypass_cache=True)
# collection.create_index('name', name='name', collation=pymongo.collation.Collation(locale='en_US', strength=1))
# collection.create_index([('xrefs.id', pymongo.ASCENDING), ('xrefs.db', pymongo.ASCENDING)],
#                         name='xrefs',
#                         collation=pymongo.collation.Collation(locale='en_US', strength=1))
# print("Success")

Success
CPU times: user 4.33 s, sys: 338 ms, total: 4.67 s
Wall time: 7.56 s
